# Cross Validation
## 下記理由よりCross Validationする必要がある。
- ## submit回数に制限がある。
- ## publicの検証データで良い結果(submitで出るスコア)が出ても、privateの検証データ(最終的なランキングやメダル認定に用いるデータ)で良い結果が出るとは限らない。→publicの検証データに過学習しているかもしれない。

# ホールドアウト検証（以前のコード）

In [1]:
import numpy as np
import pandas as pd


train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [2]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,0
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,0
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1


In [3]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [4]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,3,0,22.0,1,0,7.2500,0,2,0
1,1,1,38.0,1,0,71.2833,1,2,0
2,3,1,26.0,0,0,7.9250,0,1,1
3,1,1,35.0,1,0,53.1000,0,2,0
4,3,0,35.0,0,0,8.0500,0,1,1


### ホールドアウト検証(以前のコードのLightGBMの部分)

In [5]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)

# ハイパーパラメータは手動調整済み
categorical_features = ['Embarked', 'Pclass', 'Sex']
params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train,
                                         categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                                        categorical_feature=categorical_features)

model = lgb.train(params, lgb_train,
                               valid_sets=[lgb_train, lgb_eval],
                               verbose_eval=10,
                               num_boost_round=1000,
                               early_stopping_rounds=10)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.505699	valid_1's binary_logloss: 0.532106
[20]	training's binary_logloss: 0.427825	valid_1's binary_logloss: 0.482279
[30]	training's binary_logloss: 0.377242	valid_1's binary_logloss: 0.456641
[40]	training's binary_logloss: 0.345424	valid_1's binary_logloss: 0.447083
[50]	training's binary_logloss: 0.323113	valid_1's binary_logloss: 0.440407
[60]	training's binary_logloss: 0.302727	valid_1's binary_logloss: 0.434527
[70]	training's binary_logloss: 0.285597	valid_1's binary_logloss: 0.434932
Early stopping, best iteration is:
[66]	training's binary_logloss: 0.293072	valid_1's binary_logloss: 0.433251


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [6]:
y_pred[:10]

array([0.03605598, 0.40306884, 0.10732166, 0.0802399 , 0.46011271,
       0.20222002, 0.64929492, 0.11896033, 0.7452973 , 0.01917651])

In [7]:
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [8]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')

sub['Survived'] = y_pred
sub.to_csv('submission_lightgbm_holdout.csv', index=False)

sub.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


## Cross Validation

In [9]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,3,0,22.0,1,0,7.2500,0,2,0
1,1,1,38.0,1,0,71.2833,1,2,0
2,3,1,26.0,0,0,7.9250,0,1,1
3,1,1,35.0,1,0,53.1000,0,2,0
4,3,0,35.0,0,0,8.0500,0,1,1


### KFold
KFoldを用いる事でtrain_test_split()を複数回用いて、交差検証用のデータを作らなくてよくなる

In [10]:
from sklearn.model_selection import KFold

# 各分割でのX_testに対する予測値を格納するリスト
y_preds = []
# 各分割での学習したモデルを格納するリスト
models = []
# 各分割での検証用データセットに予測値を格納するリスト（0で初期化）
# oof(out of fold)：交差検証用において、学習用データを、さらに学習用データと検証用データに分割した時に、学習に使われなかったデータ
oof_train = np.zeros((len(X_train),))

# n_splits:分割数
# shuffle：分割前にデータをシャッフルするかどうか
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = ['Embarked', 'Pclass', 'Sex']

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

# fold_idは分割して学習したモデルを個別に保存する場合にファイル名に用いられることが多い（今回は未使用）
# 分割後のイテラブルオブジェクトはインデックスなので、データを抽出しなければならないことに注意、インデックスの指定方法もDFとndaaryで異なる
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    # データフレーム
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    # numpy.ndarray
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,
                                            categorical_feature=categorical_features)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    # CrossValidationの検証用データセット（X_val）に対する予測値を格納(冒頭で0で初期化したリストに代入)
    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    
    # 各分割でのX_test（最初に用意した評価用のデータセット）に対する予測値を格納
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_preds.append(y_pred)
    
    # 各分割で学習したモデルを格納
    models.append(model)

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.506339	valid_1's binary_logloss: 0.516198
[20]	training's binary_logloss: 0.428668	valid_1's binary_logloss: 0.446596
[30]	training's binary_logloss: 0.383412	valid_1's binary_logloss: 0.411961
[40]	training's binary_logloss: 0.35367	valid_1's binary_logloss: 0.397122
[50]	training's binary_logloss: 0.329451	valid_1's binary_logloss: 0.391041
[60]	training's binary_logloss: 0.307092	valid_1's binary_logloss: 0.38325
[70]	training's binary_logloss: 0.290771	valid_1's binary_logloss: 0.377067
[80]	training's binary_logloss: 0.274935	valid_1's binary_logloss: 0.373918
[90]	training's binary_logloss: 0.260335	valid_1's binary_logloss: 0.370602
Early stopping, best iteration is:
[85]	training's binary_logloss: 0.267203	valid_1's binary_logloss: 0.369116
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.504211	valid_1's binary_logloss: 0.534594
[20]	trai

In [11]:
# oof_trainをcsv形式で保存　（事前にndarrayからdfに変換しておく）
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

各モデルの検証用データセットに対する性能(損失関数の値)を表示、CVスコアとして平均も表示

In [12]:
scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

===CV scores===
[0.3691161193267496, 0.4491122965802196, 0.3833384988458873, 0.43712149656630833, 0.43469994547894103]
0.41467767135962114


性能を、正解率で表示

In [13]:
from sklearn.metrics import accuracy_score


y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

0.8226711560044894

各分割での予測値を平均した値を、最終的なsubmitとする

In [14]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
sub['Survived'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False)
sub.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


# 結果の比較
## 交差検証を用いると精度が向上したことが分かる
- クロスバリデーション未使用時：0.76315
- クロスバリデーション適用時：0.76794